In [2]:
from pprint import pprint

from modelconstruct import Mexplode
from modelpattern import list_extract

No modelwidgets


In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
eq = '''
list ages = ages : age_0 * age_11 
            
list sexes = sexes : female male
£ primo population is (ultimo)population in age group before
doable [ages=ages_nostart] pop_primo__a = pop__ab(-1)

£ for first age group the population depends on all births 
doable [ages=ages_start]   pop_before_death__a   = birth__total + migration__a
doable [ages=ages_nostart] pop_before_death__a   = pop_primo__a + migration__a 

£ --- Step 3: Deaths ---
doable  [ages=ages_end ] death_rate__a =1.0 
doable  <sum=total>          death__a = pop_before_death__a  * death_rate__a

£ --- Step 4: End-of-period population ---
doable  <sum=total>          pop__a  = pop_before_death__a  - death__a 

£ --- Step 5: Births from fertile ages (all except AGE_0) ---
doable <sum=total> [ages=ages_nostart]        birth__a = pop_primo__a * fertility__a

£ --- Step 6: Totals and consistency check ---
migration__total = sum(ages,migration__a)
pop__total_check  = (pop__total(-1) + birth__total + migration__total - death__total) - pop__total


'''
print('This Input:')
(mm := Mexplode(eq,replacements=[('__ab','__{ages_before}'),('__a','__{ages}')])) #.showclean_frml_statements  
print('Creates this Output')
mm.show

This Input:
Creates this Output
FRML <> POP_PRIMO__AGE_1 = POP__AGE_0(-1) $
FRML <> POP_PRIMO__AGE_2 = POP__AGE_1(-1) $
FRML <> POP_PRIMO__AGE_3 = POP__AGE_2(-1) $
FRML <> POP_PRIMO__AGE_4 = POP__AGE_3(-1) $
FRML <> POP_PRIMO__AGE_5 = POP__AGE_4(-1) $
FRML <> POP_PRIMO__AGE_6 = POP__AGE_5(-1) $
FRML <> POP_PRIMO__AGE_7 = POP__AGE_6(-1) $
FRML <> POP_PRIMO__AGE_8 = POP__AGE_7(-1) $
FRML <> POP_PRIMO__AGE_9 = POP__AGE_8(-1) $
FRML <> POP_PRIMO__AGE_10 = POP__AGE_9(-1) $
FRML <> POP_BEFORE_DEATH__AGE_0 = BIRTH__TOTAL+MIGRATION__AGE_0 $
FRML <> POP_BEFORE_DEATH__AGE_1 = POP_PRIMO__AGE_1+MIGRATION__AGE_1 $
FRML <> POP_BEFORE_DEATH__AGE_2 = POP_PRIMO__AGE_2+MIGRATION__AGE_2 $
FRML <> POP_BEFORE_DEATH__AGE_3 = POP_PRIMO__AGE_3+MIGRATION__AGE_3 $
FRML <> POP_BEFORE_DEATH__AGE_4 = POP_PRIMO__AGE_4+MIGRATION__AGE_4 $
FRML <> POP_BEFORE_DEATH__AGE_5 = POP_PRIMO__AGE_5+MIGRATION__AGE_5 $
FRML <> POP_BEFORE_DEATH__AGE_6 = POP_PRIMO__AGE_6+MIGRATION__AGE_6 $
FRML <> POP_BEFORE_DEATH__AGE_7 = POP_PRI

In [6]:
mm.showclean_frml_statements 

Clean_frml_statements: 
LIST AGES =
    AGES : AGE_0 * AGE_11  $

LIST SEXES =
    SEXES : FEMALE MALE  $
£ PRIMO POPULATION IS (ULTIMO)POPULATION IN AGE GROUP BEFORE
DOABLE [AGES=AGES_NOSTART] POP_PRIMO__{AGES} = POP__{AGES_BEFORE}(-1)$

£ FOR FIRST AGE GROUP THE POPULATION DEPENDS ON ALL BIRTHS 
DOABLE [AGES=AGES_START]   POP_BEFORE_DEATH__{AGES}   = BIRTH__TOTAL + MIGRATION__{AGES}$
DOABLE [AGES=AGES_NOSTART] POP_BEFORE_DEATH__{AGES}   = POP_PRIMO__{AGES} + MIGRATION__{AGES} $

£ --- STEP 3: DEATHS ---
DOABLE  [AGES=AGES_END ] DEATH_RATE__{AGES} =1.0 $
DOABLE  <SUM=TOTAL>          DEATH__{AGES} = POP_BEFORE_DEATH__{AGES}  * DEATH_RATE__{AGES}$

£ --- STEP 4: END-OF-PERIOD POPULATION ---
DOABLE  <SUM=TOTAL>          POP__{AGES}  = POP_BEFORE_DEATH__{AGES}  - DEATH__{AGES} $

£ --- STEP 5: BIRTHS FROM FERTILE AGES (ALL EXCEPT AGE_0) ---
DOABLE <SUM=TOTAL> [AGES=AGES_NOSTART]        BIRTH__{AGES} = POP_PRIMO__{AGES} * FERTILITY__{AGES}$

£ --- STEP 6: TOTALS AND CONSISTENCY CHECK ---
F

In [7]:
mm.mmodel.pop__age_2.tracepre(up=6,size=(10,10))

In [8]:
import pandas as pd

# --- model structure ---
ages = [f'AGE_{i}' for i in range(11)]  # AGE_0 ... AGE_10
variables = [f'POP__{age}' for age in ages]

years = range(2010, 2031)  # example period (2010–2030)

# --- initial population distribution (illustrative numbers, can be replaced) ---
# e.g. a pyramid decreasing by age
base_population = {
    f'POP__AGE_0':  400000,
    f'POP__AGE_1':  390000,
    f'POP__AGE_2':  385000,
    f'POP__AGE_3':  380000,
    f'POP__AGE_4':  370000,
    f'POP__AGE_5':  360000,
    f'POP__AGE_6':  340000,
    f'POP__AGE_7':  320000,
    f'POP__AGE_8':  300000,
    f'POP__AGE_9':  270000,
    f'POP__AGE_10': 240000,
}

# --- create the DataFrame ---
df = pd.DataFrame(index=years, columns=variables, dtype=float)

# fill only the first year (e.g. 2010)
start_year = 2010
for var, value in base_population.items():
    df.loc[start_year, var] = value

# optional: forward-fill or leave NaN for model simulation
df = df.ffill()

df.head()


,POP__AGE_0,POP__AGE_1,POP__AGE_2,POP__AGE_3,POP__AGE_4,POP__AGE_5,POP__AGE_6,POP__AGE_7,POP__AGE_8,POP__AGE_9,POP__AGE_10
2010,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,240000.0
2011,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,240000.0
2012,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,240000.0
2013,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,240000.0
2014,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,240000.0


In [9]:
import pandas as pd
import numpy as np

years = range(2010, 2031)  # same period as your population DataFrame
ages = [f'AGE_{i}' for i in range(11)]

# --------------------
# 1. DEATH RATES (by age)
# --------------------
death_rates = {
    'AGE_0': 0.010,  # infant mortality
    'AGE_1': 0.001,
    'AGE_2': 0.0008,
    'AGE_3': 0.0007,
    'AGE_4': 0.0008,
    'AGE_5': 0.0010,
    'AGE_6': 0.0020,
    'AGE_7': 0.0040,
    'AGE_8': 0.0100,
    'AGE_9': 0.0200,
    # AGE_10 assumed handled in model (set = 1.0)
}

death_rate_vars = [f'DEATH_RATE__{age}' for age in ages[:-1]]  # AGE_0–AGE_9
df_death = pd.DataFrame(index=years, columns=death_rate_vars, dtype=float)
for var in death_rate_vars:
    df_death[var] = death_rates[var.split('__')[-1]]

# --------------------
# 2. FERTILITY RATES (fertile ages only)
# --------------------
fertility_rates = {
    'AGE_1': 0.000,  # not fertile
    'AGE_2': 0.010,
    'AGE_3': 0.040,
    'AGE_4': 0.070,
    'AGE_5': 0.080,
    'AGE_6': 0.060,
    'AGE_7': 0.030,
    'AGE_8': 0.010,
    'AGE_9': 0.002,
    'AGE_10': 0.000,
}

fertility_vars = [f'FERTILITY__{age}' for age in ages[1:]]  # AGE_1–AGE_10
df_fert = pd.DataFrame(index=years, columns=fertility_vars, dtype=float)
for var in fertility_vars:
    df_fert[var] = fertility_rates[var.split('__')[-1]]

# --------------------
# 3. MIGRATION (net migration, can be + or −)
# --------------------
migration_pattern = {
    'AGE_0':  500,   # babies moving with parents
    'AGE_1':  400,
    'AGE_2':  200,
    'AGE_3':  100,
    'AGE_4':   50,
    'AGE_5':  -50,
    'AGE_6': -100,
    'AGE_7': -200,
    'AGE_8': -300,
    'AGE_9': -400,
    'AGE_10': -500,
}

migration_vars = [f'MIGRATION__{age}' for age in ages]
df_mig = pd.DataFrame(index=years, columns=migration_vars, dtype=float)
for var in migration_vars:
    df_mig[var] = migration_pattern[var.split('__')[-1]]

# --------------------
# 4. Combine all
# --------------------
df_exo = pd.concat([df_death, df_fert, df_mig], axis=1)
df_exo.index.name = 'year'

df_exo.head()


,DEATH_RATE__AGE_0,DEATH_RATE__AGE_1,DEATH_RATE__AGE_2,DEATH_RATE__AGE_3,DEATH_RATE__AGE_4,DEATH_RATE__AGE_5,DEATH_RATE__AGE_6,DEATH_RATE__AGE_7,DEATH_RATE__AGE_8,DEATH_RATE__AGE_9,...,MIGRATION__AGE_1,MIGRATION__AGE_2,MIGRATION__AGE_3,MIGRATION__AGE_4,MIGRATION__AGE_5,MIGRATION__AGE_6,MIGRATION__AGE_7,MIGRATION__AGE_8,MIGRATION__AGE_9,MIGRATION__AGE_10
year,,,,,,,,,,,,,,,,,,,,,
2010,0.01,0.001,0.0008,0.0007,0.0008,0.001,0.002,0.004,0.01,0.02,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2011,0.01,0.001,0.0008,0.0007,0.0008,0.001,0.002,0.004,0.01,0.02,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2012,0.01,0.001,0.0008,0.0007,0.0008,0.001,0.002,0.004,0.01,0.02,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2013,0.01,0.001,0.0008,0.0007,0.0008,0.001,0.002,0.004,0.01,0.02,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2014,0.01,0.001,0.0008,0.0007,0.0008,0.001,0.002,0.004,0.01,0.02,...,400,200,100,50,-50,-100,-200,-300,-400,-500


In [10]:
df_full = pd.concat([df, df_exo], axis=1)
df_full

,POP__AGE_0,POP__AGE_1,POP__AGE_2,POP__AGE_3,POP__AGE_4,POP__AGE_5,POP__AGE_6,POP__AGE_7,POP__AGE_8,POP__AGE_9,...,MIGRATION__AGE_1,MIGRATION__AGE_2,MIGRATION__AGE_3,MIGRATION__AGE_4,MIGRATION__AGE_5,MIGRATION__AGE_6,MIGRATION__AGE_7,MIGRATION__AGE_8,MIGRATION__AGE_9,MIGRATION__AGE_10
2010,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2011,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2012,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2013,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2014,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2015,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2016,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2017,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2018,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500
2019,400000.0,390000.0,385000.0,380000.0,370000.0,360000.0,340000.0,320000.0,300000.0,270000.0,...,400,200,100,50,-50,-100,-200,-300,-400,-500


In [11]:
mpop = mm.mmodel
mpop(df_full)

Will start calculating: testmodel
2011  solved
2012  solved
2013  solved
2014  solved
2015  solved
2016  solved
2017  solved
2018  solved
2019  solved
2020  solved
2021  solved
2022  solved
2023  solved
2024  solved
2025  solved
2026  solved
2027  solved
2028  solved
2029  solved
2030  solved
testmodel calculated 


,POP__AGE_0,POP__AGE_1,POP__AGE_2,POP__AGE_3,POP__AGE_4,POP__AGE_5,POP__AGE_6,POP__AGE_7,POP__AGE_8,POP__AGE_9,...,POP__TOTAL,POP_PRIMO__AGE_4,POP_PRIMO__AGE_6,DEATH__AGE_6,BIRTH__AGE_9,DEATH__AGE_7,POP_BEFORE_DEATH__AGE_7,POP__TOTAL_CHECK,DEATH__AGE_5,BIRTH__AGE_10
2010,400000.000000,390000.000000,385000.000000,380000.000000,370000.000000,360000.000000,340000.000000,320000.000000,300000.000000,270000.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0
2011,110484.000000,399999.600000,389887.840000,384830.430000,379745.960000,369580.050000,359180.200000,338440.800000,316503.000000,293608.000000,...,3.342260e+06,380000.000000,360000.000000,719.800000,600.000000,1359.200000,339800.000000,-3.515000e+06,369.950000,0.0
2012,113237.130105,110773.116000,399879.440320,389714.848512,384572.525656,379316.264040,368741.089900,357544.279200,334759.392000,309780.940000,...,3.148319e+06,384830.430000,369580.050000,738.960100,633.006000,1435.920800,358980.200000,-4.656613e-10,379.695960,0.0
2013,112577.771133,113523.492975,110884.337507,399699.454712,389453.036633,384138.003130,378457.831512,367066.925540,353671.836408,327672.204160,...,2.937145e+06,389714.848512,379316.264040,758.432528,669.518784,1474.164360,368541.089900,4.656613e-10,384.522526,0.0
2014,102945.982839,112864.793362,113632.514181,110906.648471,399429.655148,389013.633597,383269.927124,376744.800186,363099.256285,346206.399680,...,2.698114e+06,399699.454712,384138.003130,768.076006,707.343673,1513.031326,378257.831512,-4.656613e-10,389.403037,0.0
2015,84372.105163,103242.636857,112974.341527,113652.901421,110867.883152,398980.275493,388135.806329,381537.647416,372680.352184,355445.271159,...,2.421889e+06,110906.648471,389013.633597,777.827267,726.198513,1532.279708,383069.927124,4.656613e-10,399.379655,0.0
2016,62389.960062,84687.333058,103359.882747,112995.189488,113611.939099,110707.065269,398082.514942,386384.063104,377425.270941,364834.745140,...,2.114478e+06,113652.901421,398980.275493,797.760551,745.360704,1551.743225,387935.806329,-4.656613e-10,110.817883,0.0
2017,45226.642794,62727.170102,84819.423191,103387.460829,112954.753337,113448.377160,110385.851138,396290.984882,382223.222473,369484.765523,...,1.780949e+06,112995.189488,110707.065269,221.214131,754.850542,1591.530060,397882.514942,0.000000e+00,113.561939,0.0
2018,35282.991755,45581.016151,62876.828366,84859.979595,103354.710860,112791.848583,113121.680406,109745.107734,392031.075033,374186.758024,...,1.433832e+06,103387.460829,113448.377160,226.696754,764.446445,440.743405,110185.851138,0.000000e+00,112.904753,0.0
2019,29424.911958,35647.308763,45744.391339,62932.744586,84842.051612,103201.406150,112466.464886,112469.993684,108350.656657,383798.453533,...,1.078878e+06,84859.979595,112791.848583,225.383697,784.062150,451.686722,112921.680406,-2.328306e-10,103.304711,0.0


In [12]:
with mpop.set_smpl(2010,2030):
    print(mpop['pop__total death__total birth__total'].df)

        POP__TOTAL   DEATH__TOTAL   BIRTH__TOTAL
2010  0.000000e+00       0.000000       0.000000
2011  3.342260e+06  283540.120000  111100.000000
2012  3.148319e+06  307521.793767  113880.939500
2013  2.937145e+06  324089.052359  113214.920337
2014  2.698114e+06  342217.124091  103485.841252
2015  2.421889e+06  360648.738821   84724.348650
2016  2.114478e+06  369631.418528   62520.161679
2017  1.780949e+06  378412.789991   45183.477570
2018  1.433832e+06  381956.040533   35139.385611
2019  1.078878e+06  383875.746632   29222.133291
2020  7.151567e+05  387708.071200   24286.384557
2021  6.247652e+05  109573.245632   19481.708048
2022  5.277275e+05  111910.185962   15172.531035
2023  4.285485e+05  110723.550127   11844.589562
2024  3.369313e+05  100761.305180    9444.066622
2025  2.620516e+05   82239.454252    7659.699868
2026  2.073452e+05   60634.966446    6228.638800
2027  1.682349e+05   43848.935775    5038.579860
2028  1.379120e+05   34091.588136    4068.736661
2029  1.126332e+05  

In [14]:
mm.list_specification

'LIST AGES =\n    AGES : AGE_0 * AGE_11 $\nLIST SEXES =\n    SEXES : FEMALE MALE $'